In [1]:
# import QGRID para mejor visualizacion de los datos
# los pptos sin abonos se deben cancelar en ClinicCloud, cosa q no generen alertas. O q el paciente tenga saldo a favor
# Debemos Generar un archivo identificandolos. Las ff´s deben tener algun ingreso de dinero para no cancelarlas.
# Los pptos aceptados hoy pueden haber sido elaborados en una fecha anterior. Su fecha se debe actualizar a la de hoy, 
# ya sea en ClinicCloud o en el reporte (mejor - 'fecha_inicio')

import numpy as np
import pandas as pd
import time

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', -1)

In [2]:
archivo_PPE = 'Produccion_pendiente_exigible_con_tratamientos_Dentix_Calle_90.csv'
archivo_crono = 'crono_Dentix_Calle_90_1.csv'

In [3]:
crono_df = pd.read_csv(archivo_crono, encoding = 'unicode_escape', sep=',')
PPE = pd.read_csv(archivo_PPE, encoding = 'unicode_escape', sep=',')

#Eliminación de Puntos y Revisiones del PPE
PPE.set_index('Servicio', inplace=True)
PPE.drop(index='Puntos y Revisiones.  ',inplace=True)
PPE.reset_index(inplace=True)
PPE['duplicated'] = PPE.duplicated()
PPE.set_index('duplicated',inplace=True)
PPE.drop(True,inplace=True)
PPE.reset_index(inplace=True)
PPE.drop(columns=['duplicated'],inplace=True)
PPE

,Servicio,Clínica,Fecha Pres.,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor,Especialidad,Explicado,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod
0,BOCA SANA.,Dentix Calle 90,12/09/2018,BG056348,Nº 4425 - rodrigo ramos urrego,0,3123979127,LAURA ALEJANDRA RONCANCIO GARCIA,Primeras Visitas,ALISSON SARAY MORA CASTAÑEDA,0,0,100%,0.00%,0,0.0,15/09/2018,Sin Cita Producida
1,PROFILAXIS. Pieza: 11,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,40000,0,0.00%,0.00%,0,40000.0,13/09/2018,Sin Cita Producida
2,DETARTRAJE POR ARCADA.,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,97520,0,0.00%,0.00%,0,97500.0,13/09/2018,Sin Cita Producida
3,RESINA FOTOPOLIMERIZADO 2 SUP POSTERIOR Y POSTERIOR. Pieza: 11,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,100000,100000,100%,100.00%,100000,0.0,13/09/2018,12/09/2018
4,CORONA SOBRE IMPLANTE. Pieza: 36,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,REHABILITACIÓN,SANDRA PATRICIA GONZALEZ TOVAR,1385650,0,0.00%,0.00%,0,1385650.0,13/09/2018,Sin Cita Producida
5,PLACA DE BRUXISMO. Pieza: 11,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,265000,0,0.00%,0.00%,0,265000.0,13/09/2018,Sin Cita Producida
6,CARILLA ESTETICA EN RESINA. Pieza: 11,Dentix Calle 90,12/09/2018,BG056521,Nº 4448 - VIANI ELIZA DIAZ,0,3005771589,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,304950,0,0.00%,0.00%,0,304950.0,13/09/2018,Sin Cita Producida
7,CARILLA ESTETICA EN RESINA. Pieza: 12,Dentix Calle 90,12/09/2018,BG056521,Nº 4448 - VIANI ELIZA DIAZ,0,3005771589,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,304950,0,0.00%,0.00%,0,304950.0,13/09/2018,Sin Cita Producida
8,DETARTRAJE POR ARCADA.,Dentix Calle 90,12/09/2018,BG056521,Nº 4448 - VIANI ELIZA DIAZ,0,3005771589,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,97520,0,0.00%,0.00%,0,97500.0,13/09/2018,Sin Cita Producida
9,PROFILAXIS. Pieza: 11,Dentix Calle 90,12/09/2018,BG056521,Nº 4448 - VIANI ELIZA DIAZ,0,3005771589,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,40000,0,0.00%,0.00%,0,40000.0,13/09/2018,Sin Cita Producida


In [4]:
print('REGISTROS CRONOGRAMA:',crono_df.count()[0])
print('REGISTROS PPE:',PPE.count()[0])

REGISTROS CRONOGRAMA: 547
REGISTROS PPE: 565


In [5]:
#CRUCE PPE - CRONO: en este punto revisar que cada ttmto del PPE tenga su cronograma. Sinó, sacar los pptos q faltan por crono.

pptos_faltantes = pd.merge(PPE,crono_df,how='left',left_on=['Clínica','Nº Historia','Presupuesto','Servicio'], right_on=['Clínica','Nº Historia','Presupuesto','Servicio'])
pptos_faltantes['Semana'].fillna(0,inplace=True)
pptos_faltantes.set_index('Semana',inplace=True)
pptos_faltantes = pptos_faltantes.loc[0]

pptos_faltantes

,Servicio,Clínica,Fecha Pres._x,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor_x,Especialidad,Explicado_x,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod,Fecha Pres._y,Cantidad Presupuestada,Doctor_y,Explicado_y,ttmto_homol,pieza,fecha_inicio
Semana,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,BOCA SANA.,Dentix Calle 90,12/09/2018,BG056348,Nº 4425 - rodrigo ramos urrego,0,3123979127,LAURA ALEJANDRA RONCANCIO GARCIA,Primeras Visitas,ALISSON SARAY MORA CASTAÑEDA,0,0,100%,0.00%,0,0.0,15/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,PROFILAXIS. Pieza: 11,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,40000,0,0.00%,0.00%,0,40000.0,13/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,DETARTRAJE POR ARCADA.,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,97520,0,0.00%,0.00%,0,97500.0,13/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RESINA FOTOPOLIMERIZADO 2 SUP POSTERIOR Y POSTERIOR. Pieza: 11,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,100000,100000,100%,100.00%,100000,0.0,13/09/2018,12/09/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,CORONA SOBRE IMPLANTE. Pieza: 36,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,REHABILITACIÓN,SANDRA PATRICIA GONZALEZ TOVAR,1385650,0,0.00%,0.00%,0,1385650.0,13/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,PLACA DE BRUXISMO. Pieza: 11,Dentix Calle 90,12/09/2018,BG058181,Nº 4432 - maria claudia barreto jimenez,0,3153871120,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,265000,0,0.00%,0.00%,0,265000.0,13/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,CARILLA ESTETICA EN RESINA. Pieza: 11,Dentix Calle 90,12/09/2018,BG056521,Nº 4448 - VIANI ELIZA DIAZ,0,3005771589,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,304950,0,0.00%,0.00%,0,304950.0,13/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,CARILLA ESTETICA EN RESINA. Pieza: 12,Dentix Calle 90,12/09/2018,BG056521,Nº 4448 - VIANI ELIZA DIAZ,0,3005771589,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,304950,0,0.00%,0.00%,0,304950.0,13/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,DETARTRAJE POR ARCADA.,Dentix Calle 90,12/09/2018,BG056521,Nº 4448 - VIANI ELIZA DIAZ,0,3005771589,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,97520,0,0.00%,0.00%,0,97500.0,13/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pptos_faltantes.reset_index(inplace=True)
pptos_faltantes['Cantidad Presupuestada'] = (pptos_faltantes['Producidos']+pptos_faltantes['Pendientes Producción'])/pptos_faltantes['Importe Pres.']
pptos_faltantes['Cantidad Presupuestada'] = np.ceil(pptos_faltantes['Cantidad Presupuestada'])
pptos_faltantes['Cantidad Presupuestada'].fillna(1,inplace=True) #las CSI traen vacío el campo 'Pendientes Produccion'.
# en realidad parece que las unicas cantidades presupuestadas q son necesarias son las de Mensualidades de ortodoncia.
# Para CSI, el %Realizado es el que nos sirve para calcular Cantidad Producida.
# la Cantidad Pendiente se puede utilizar para saber la fecha de colocación de un implante si hay ortodoncia.

pptos_faltantes['ttmto_homol'] = pptos_faltantes['Servicio'].str.slice(0,-13)
pptos_faltantes['ttmto_homol'] = pptos_faltantes['ttmto_homol'].str.strip()
pptos_faltantes['pieza'] = pptos_faltantes['Servicio'].str.slice(-2)
pptos_faltantes['pieza'] = pptos_faltantes['pieza'].str.strip()
pptos_faltantes['fecha_inicio'] = time.strftime("%d/%m/%Y")

pptos_faltantes = pptos_faltantes[['Clínica', 'Fecha Pres._x', 'Nº Historia', 'Presupuesto', 
                                   'Servicio', 'Semana', 'Cantidad Presupuestada','Doctor_x', 
                                   'Explicado_x', 'ttmto_homol','pieza', 'fecha_inicio']]

print('Numero de tratamientos faltantes por crono:',pptos_faltantes.count()[0])
pptos_faltantes.to_csv('pptos_faltantes.csv',index = False)

pptos_faltantes.set_index(['Clínica','Nº Historia','Servicio'], inplace=True)
pptos_faltantes

Numero de tratamientos faltantes por crono: 65


Fecha Pres._x  \
Clínica         Nº Historia Servicio                                                                            
Dentix Calle 90 BG056348    BOCA SANA.                                                           12/09/2018     
                BG058181    PROFILAXIS.   Pieza: 11                                              12/09/2018     
                            DETARTRAJE POR ARCADA.                                               12/09/2018     
                            RESINA FOTOPOLIMERIZADO 2 SUP POSTERIOR Y POSTERIOR.   Pieza: 11     12/09/2018     
                            CORONA SOBRE IMPLANTE.   Pieza: 36                                   12/09/2018     
                            PLACA DE BRUXISMO.   Pieza: 11                                       12/09/2018     
                BG056521    CARILLA ESTETICA EN RESINA.   Pieza: 11                              12/09/2018     
                            CARILLA ESTETICA EN RESINA.   Pieza: 12                              12/09/2018     
                            DETARTRAJE POR ARCADA.                                               12/09/2018     
                            PROFILAXIS.   Pieza: 11                                              12/09/2018     
                            BLANQUEAMIENTO DENTAL 1.   Pieza: 11                                 12/09/2018     
                BG057147    DETARTRAJE POR ARCADA.                                               12/09/2018     
                            PROFILAXIS.   Pieza: 11                                              12/09/2018     
                            BLANQUEAMIENTO DENTAL 1.   Pieza: 11                                 12/09/2018     
                BG058106    CORONA METAL CERAMICA.   Pieza: 13                                   12/09/2018     
                BG058184    PROFILAXIS.   Pieza: 11                                              12/09/2018     
                            DETARTRAJE POR ARCADA.                                               12/09/2018     
                            ESTUDIO CEFALOMETRICO.   Pieza: 11                                   12/09/2018     
                            ENTRADA BRACKETS ARCADA INFERIOR ESTETICOS O CERAMICOS.   Pieza: 31  12/09/2018     
                            ENTRADA BRACKETS ARCADA SUPERIOR ESTETICOS O CERAMICOS.   Pieza: 21  12/09/2018     
                            MENSUALIDAD ORTODONCIA ESTETICOS O CERAMICOS.   Pieza: 11            12/09/2018     
                            RETENEDOR ESSIX O ESTETICOS.   Pieza: 21                             12/09/2018     
                            RETENEDOR ESSIX O ESTETICOS.   Pieza: 31                             12/09/2018     
                            RETENEDOR FIJO 3X3 EN ALAMBRE.   Pieza: 31                           12/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 17      12/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 16      12/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 26      12/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 27      12/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 36      12/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 47      12/09/2018     
                            RETIRO BRACKETS ARCADA SUPERIOR.   Pieza: 21                         12/09/2018     
                BG058185    PROFILAXIS.   Pieza: 11                                              12/09/2018     
                            DETARTRAJE POR ARCADA.                                               12/09/2018     
                            RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR.   Pieza: 36      12/09/2018     
                            